In [ ]:
# Setup to analyse an MDP Playground experiment
from mdp_playground.analysis import MDPP_Analysis

# Set the following to True to save PDFs of plots that you generate below
save_fig = False

In [ ]:
# Data loading
mdpp_analysis = MDPP_Analysis()

# load multiple experiments data

# experiments = { <exp_name_1>: <dir_name_1>, ... }
# Set dir_name to the location where the CSV files from running an experiment were saved
# Set exp_name to the name that was given to the experiment when running it
experiments = {
             "td3_move_to_a_point_action_max": "/media/suresh/work/ml_hiwi/data/mdpp_12713006",
             "sac_move_to_a_point_action_max": "/media/suresh/work/ml_hiwi/data/mdpp_12746746",
             "td3_move_to_a_point_time_unit": "/media/suresh/work/ml_hiwi/data/mdpp_12713385",
             "sac_move_to_a_point_time_unit": "/media/suresh/work/ml_hiwi/data/mdpp_12747791",
}

list_exp_data = mdpp_analysis.load_data(experiments)

In [ ]:
# 1-D: Plots showing reward after 20k timesteps when varying a single meta-feature
# Plots across 10 runs: Training: with std dev across the runs
mdpp_analysis.plot_1d_dimensions(list_exp_data, save_fig, plot_type = "agent")

In [ ]:
# Plots across 10 runs: Evaluation: with std dev across the runs
mdpp_analysis.plot_1d_dimensions(list_exp_data, save_fig, train=False, plot_type = "agent")

In [ ]:
# 1-D: Plots showing reward after 20k timesteps when varying a single meta-feature
# Plots across 10 runs: Training: with std dev across the runs
mdpp_analysis.plot_1d_dimensions(list_exp_data, save_fig, plot_type = "metric")

In [ ]:
# Plots across 10 runs: Evaluation: with std dev across the runs
mdpp_analysis.plot_1d_dimensions(list_exp_data, save_fig, train=False, plot_type = "metric")

In [ ]:
# This and the next cell do the same as the previous 2 cells but plot episode mean lengths instead of episode reward
mdpp_analysis.plot_1d_dimensions(list_exp_data, save_fig, metric_num=-1, plot_type = "agent")

In [ ]:
mdpp_analysis.plot_1d_dimensions(list_exp_data, save_fig, train=False, metric_num=-1, plot_type = "agent")

In [ ]:
# 2-D heatmap plots across 10 runs: Training runs: with std dev across the runs
# There seems to be a bug with matplotlib - x and y axes tick labels are not correctly set even though we pass them. Please feel free to look into the code and suggest a correction if you find it.
mdpp_analysis.plot_2d_heatmap(list_exp_data, save_fig)

In [ ]:
# 2-D heatmap plots across 10 runs: Evaluation runs: with std dev across the runs
mdpp_analysis.plot_2d_heatmap(list_exp_data, save_fig, train=False)

In [ ]:
# Plot learning curves: Training: Each curve corresponds to a different seed for the agent
mdpp_analysis.plot_learning_curves(list_exp_data, save_fig)

In [ ]:
# Plot learning curves: Evaluation: Each curve corresponds to a different seed for the agent
mdpp_analysis.plot_learning_curves(list_exp_data, save_fig, train=False)